In [24]:
import numpy as np
import pandas as pd

On se connecte à la base de données

In [25]:
from db_utils import get_db_engine

# Obtenir le moteur de base de données
engine = get_db_engine()

In [26]:
# Charger les fichiers sources
file1 = 'data\\ObesityDataSet_raw_and_data_sinthetic.csv'
file2 = 'data\\lifestyle_sustainability_data.csv'
file3 = 'data\\DietSurvey_toJson.json'

# Charger les fichiers dans des DataFrames
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df3 = pd.read_json(file3)

In [27]:
# habits1 - df1
habits1 = df1[['NCP']].rename(
    columns={
        'NCP': 'nbr_meal_day',
    }
)
habits1['takeout_per_week'] = None
habits1['monitor_cal'] = None

# Arrondi des valeurs dans la colonne 'nbr_meal_day' à l'entier le plus proche
habits1['nbr_meal_day'] = habits1['nbr_meal_day'].round().astype(int)


print("habits1 préparé.")

# habits2 - df2
habits2 = df2[['LocalFoodFrequency']].rename(
    columns={
        'LocalFoodFrequency': 'takeout_per_week',
    }
)
habits2['nbr_meal_day'] = None
habits2['monitor_cal'] = None

print("habits2 préparé.")

# habits3 - df3
habits3 = df3[
    [
        "How many meals do you have a day? (number of regular occasions in a day when a significant and reasonably filling amount of food is eaten)",
        "Choose all that apply: [I consult a nutritionist/dietician]",
        "How many times a week do you order-in or go out to eat?"
    ]
].rename(
    columns={
        "How many meals do you have a day? (number of regular occasions in a day when a significant and reasonably filling amount of food is eaten)":"nbr_meal_day",
        "Choose all that apply: [I consult a nutritionist/dietician]":"monitor_cal",
        "How many times a week do you order-in or go out to eat?":"takeout_per_week"
    }
)

print("habits3 préparé.")


habits1 préparé.
habits2 préparé.
habits3 préparé.


In [28]:
habits2['takeout_per_week'] = habits2['takeout_per_week'].map({
    'Always': 7,
    'Often': 5,
    'Sometimes': 3,
    'Rarely': 1
})


In [29]:
# Pour habits1
print("Valeurs uniques dans habits1 :")
for column in habits1.columns:
    print(f"Colonne '{column}':")
    print(habits1[column].unique())
    print("-" * 40)

# Pour habits2
print("Valeurs uniques dans habits2 :")
for column in habits2.columns:
    print(f"Colonne '{column}':")
    print(habits2[column].unique())
    print("-" * 40)

# Pour habits3
print("Valeurs uniques dans habits3 :")
for column in habits3.columns:
    print(f"Colonne '{column}':")
    print(habits3[column].unique())
    print("-" * 40)


Valeurs uniques dans habits1 :
Colonne 'nbr_meal_day':
[3 1 4 2]
----------------------------------------
Colonne 'takeout_per_week':
[None]
----------------------------------------
Colonne 'monitor_cal':
[None]
----------------------------------------
Valeurs uniques dans habits2 :
Colonne 'takeout_per_week':
[5 3 1 7]
----------------------------------------
Colonne 'nbr_meal_day':
[None]
----------------------------------------
Colonne 'monitor_cal':
[None]
----------------------------------------
Valeurs uniques dans habits3 :
Colonne 'nbr_meal_day':
[5 4 3 2]
----------------------------------------
Colonne 'monitor_cal':
['Never' 'Rarely' 'Often' 'Sometimes']
----------------------------------------
Colonne 'takeout_per_week':
[4 1 3 2 5 7]
----------------------------------------


In [30]:
# Combinaison des DataFrames
habits_combined = pd.concat([habits1, habits2, habits3], ignore_index=True)

# Vérification des colonnes unifiées
habits_combined.head()

,nbr_meal_day,takeout_per_week,monitor_cal
0,3,None,None
1,3,None,None
2,3,None,None
3,3,None,None
4,1,None,None


In [31]:
# Obtenir les lignes uniques
unique_rows = habits_combined.drop_duplicates()

# Afficher les lignes uniques
unique_rows


,nbr_meal_day,takeout_per_week,monitor_cal
0,3,None,None
4,1,None,None
18,4,None,None
513,2,None,None
2111,None,5,None
2112,None,3,None
2113,None,1,None
2117,None,7,None
2610,5,4,Never
2611,4,1,Rarely


In [32]:
# Supprimer les doublons pour conserver uniquement les lignes uniques
drinks_unique = habits_combined.drop_duplicates().reset_index(drop=True)

# Ajouter la colonne 'id_drinks' avec des valeurs incrémentées
drinks_unique.insert(0, 'id_habits', range(1, len(drinks_unique) + 1))

# Afficher le DataFrame modifié
drinks_unique


,id_habits,nbr_meal_day,takeout_per_week,monitor_cal
0,1,3,None,None
1,2,1,None,None
2,3,4,None,None
3,4,2,None,None
4,5,None,5,None
5,6,None,3,None
6,7,None,1,None
7,8,None,7,None
8,9,5,4,Never
9,10,4,1,Rarely


In [33]:
drinks_unique.to_sql('Habits', engine, if_exists='append', index=False)

print("Les données ont été insérées avec succès dans la table 'Diet'.")


Les données ont été insérées avec succès dans la table 'Diet'.


C:\Users\user\AppData\Local\Temp\ipykernel_21212\3473113030.py:1: UserWarning: The provided table name 'Habits' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  drinks_unique.to_sql('Habits', engine, if_exists='append', index=False)
